In [5]:
%pip install pycryptodome


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from Crypto.PublicKey import RSA

# Generate RSA key pair
key_pair = RSA.generate(2048)

# Export private key
private_key = key_pair.export_key()
with open('private_key.pem', 'wb') as private_file:
    private_file.write(private_key)

# Export public key
public_key = key_pair.publickey().export_key()
with open('public_key.pem', 'wb') as public_file:
    public_file.write(public_key)

In [7]:
import os
import base64
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA

def encrypt_message(message, key):
    # Create an AES cipher object with the key and CTR mode
    aes_cipher = AES.new(key, AES.MODE_CTR)

    # Encrypt the message
    ciphertext = aes_cipher.encrypt(message)
    return ciphertext, aes_cipher.nonce

def export_rsa_encrypted_key(aes_key, rsa_public_key, output_file):
    # Create a PKCS1_OAEP cipher object with the RSA public key
    rsa_cipher = PKCS1_OAEP.new(rsa_public_key)

    # Encrypt the AES key using RSA encryption
    encrypted_key = rsa_cipher.encrypt(aes_key)

    # Write the encrypted key to the output file
    with open(output_file, 'wb') as file:
        file.write(encrypted_key)

# Person A inputs the message
message = input("Enter the message: ").encode()

# Generate AES key
aes_key = get_random_bytes(32)

# Encrypt the message using AES in CTR mode
ciphertext, nonce = encrypt_message(message, aes_key)

# Export the encrypted AES key using RSA
rsa_public_key = RSA.import_key(open('public_key.pem').read())
export_rsa_encrypted_key(aes_key, rsa_public_key, 'encrypted_key.bin')

# Export the ciphertext and nonce to files
with open('ciphertext.bin', 'wb') as file:
    file.write(ciphertext)

with open('nonce.bin', 'wb') as file:
    file.write(nonce)

# Print the encrypted ciphertext and encoded AES key
print("Encrypted Ciphertext:", base64.b64encode(ciphertext).decode())
print("Encoded AES Key:", base64.b64encode(aes_key).decode())


KeyboardInterrupt: Interrupted by user

In [ ]:
import base64
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA

def import_rsa_private_key(private_key_file):
    # Import the RSA private key from a file
    private_key = RSA.import_key(open(private_key_file).read())
    return private_key

def decrypt_aes_key(encrypted_key, rsa_private_key):
    # Create a PKCS1_OAEP cipher object with the RSA private key
    rsa_cipher = PKCS1_OAEP.new(rsa_private_key)

    # Decrypt the AES key using RSA decryption
    aes_key = rsa_cipher.decrypt(encrypted_key)

    return aes_key

def decrypt_message(ciphertext, key, nonce):
    # Create an AES cipher object with the key, CTR mode, and nonce
    aes_cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)

    # Decrypt the ciphertext
    decrypted_message = aes_cipher.decrypt(ciphertext)

    return decrypted_message

# Person B imports the encrypted AES key
encrypted_key = open('encrypted_key.bin', 'rb').read()

# Person B imports the private key
private_key = import_rsa_private_key('private_key.pem')

# Decrypt the AES key using RSA private key
aes_key = decrypt_aes_key(encrypted_key, private_key)

# Person B decrypts the ciphertext using the AES key
ciphertext = open('ciphertext.bin', 'rb').read()
nonce = open('nonce.bin', 'rb').read()

decrypted_message = decrypt_message(ciphertext, aes_key, nonce)

print("Decrypted Message:", decrypted_message.decode())


Decrypted Message: 53
